# Inspect FIC cards Data

In [65]:
import pdfplumber
from pyzbar import pyzbar
import json

from bs4 import BeautifulSoup
import re

from pathlib import Path

ROOT = Path('../')

filename = 'TESSERE scp 2025.pdf'

In [2]:
def decode_qr_code(page):
    qr = page.crop((260, 470, 420, 610), relative=True)
    qr_meta = pyzbar.decode(qr.to_image().original)[0]

    return qr_meta.data.decode('UTF-8')

def extract_athlete_info(pdf_page):
    info_list = pdf_page.extract_text().split('\n')
    qr_url = decode_qr_code(pdf_page)

    uid = info_list[2].split()[-1]  # Use the badge number as identifier

    info = {
        "card_number": uid,
        "name": info_list[4].title(),
        "categories": info_list[6].split('/'),
        "validity": info_list[10].split()[-1] ,
        "qr": qr_url,
    }

    return info, uid

In [7]:
json_data = {}
with pdfplumber.open(ROOT / 'data/external' / filename) as pdf:
    for page in pdf.pages:
        info_dict, uid = extract_athlete_info(page)
        json_data[uid] = info_dict

CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, def

In [8]:
# Save data
with open(ROOT / 'data/athletes' / 'athletes.json', 'w') as f:
    json.dump(json_data, f)